# Import Básicos

In [117]:
import requests
import os
import pandas as pd
import zipfile

#Descarga del Dataset

In [118]:
csv_url = "https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv"

In [119]:
ruta_carpeta = "/content/CSV's"

if not os.path.isdir(ruta_carpeta):
  os.mkdir(ruta_carpeta)

In [120]:
nombre_arch = ruta_carpeta + '/population.zip'


if not os.path.isfile(nombre_arch):

  arch_csv = requests.get(csv_url, stream=True)

  with open(nombre_arch,"wb") as csv:
      for chunk in arch_csv.iter_content(chunk_size=1024):
    
          if chunk:
              csv.write(chunk)

# Descomprimimos el Dataset

In [121]:
#   Descomprimimos el archivo utilizando la libreria zipfile
nombre_dir_csv = ruta_carpeta + '/population'

with zipfile.ZipFile(file=nombre_arch, mode='r') as f:
  f.extractall(path=nombre_dir_csv)

#Cargamos el Dataset

In [122]:
path_csv = "/content/CSV's/population/API_SP.POP.TOTL_DS2_en_csv_v2_3731322.csv"
df_pob = pd.read_csv(filepath_or_buffer=path_csv, skiprows=4)

In [123]:
df_pob.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,"54,208.0","55,434.0","56,234.0","56,699.0","57,029.0","57,357.0",...,"102,565.0","103,165.0","103,776.0","104,339.0","104,865.0","105,361.0","105,846.0","106,310.0","106,766.0",NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,"130,836,765.0","134,159,786.0","137,614,644.0","141,202,036.0","144,920,186.0","148,769,974.0",...,"547,482,863.0","562,601,578.0","578,075,373.0","593,871,847.0","609,978,946.0","626,392,880.0","643,090,131.0","660,046,272.0","677,243,299.0",NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,"8,996,967.0","9,169,406.0","9,351,442.0","9,543,200.0","9,744,772.0","9,956,318.0",...,"31,161,378.0","32,269,592.0","33,370,804.0","34,413,603.0","35,383,028.0","36,296,111.0","37,171,922.0","38,041,757.0","38,928,341.0",NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,"96,396,419.0","98,407,221.0","100,506,960.0","102,691,339.0","104,953,470.0","107,289,875.0",...,"370,243,017.0","380,437,896.0","390,882,979.0","401,586,651.0","412,551,299.0","423,769,930.0","435,229,381.0","446,911,598.0","458,803,476.0",NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,"5,454,938.0","5,531,451.0","5,608,499.0","5,679,409.0","5,734,995.0","5,770,573.0",...,"25,107,925.0","26,015,786.0","26,941,773.0","27,884,380.0","28,842,482.0","29,816,769.0","30,809,787.0","31,825,299.0","32,866,268.0",NaN


#Transformamos el Dataset

In [124]:
df_pob.drop(
    columns= ['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 65'],
    inplace= True
)

df_pob = pd.melt(
    df_pob, 
    id_vars=['Country Name'],
    var_name='Year',
    value_name= 'Pob'
)

df_pob.rename(
    columns={'Country Name':'Country'}, 
    inplace=True
)

In [125]:
pd.options.display.float_format = '{:,.1f}'.format
df_pob.head(5)

,Country,Year,Pob
0,Aruba,1960,"54,208.0"
1,Africa Eastern and Southern,1960,"130,836,765.0"
2,Afghanistan,1960,"8,996,967.0"
3,Africa Western and Central,1960,"96,396,419.0"
4,Angola,1960,"5,454,938.0"


# Hacemos las conversiones correspondientes

In [126]:
df_pob.dtypes

Country     object
Year        object
Pob        float64
dtype: object

In [127]:
#   Hacemos que la columna sea de tipo Categorical y los datos sean de tipo String
df_pob['Year'] = pd.Categorical(
    df_pob['Year'].apply(str)
)

In [128]:
df_pob.dtypes

Country      object
Year       category
Pob         float64
dtype: object

# Creamos un SubSet

In [129]:
#   Hacemos una mascara booleanas que contengan los valores de los paises selecionados

idx_filtro = df_pob['Country'].isin(['Mexico', 'Hong Kong SAR, China', 'Ireland'])

In [130]:
#   Hacemos un subconjunto utilizando la mascara booleana que contenga unicamente
#   los paises seleccionados   

df_ejemplo = df_pob[idx_filtro]

In [131]:
df_ejemplo

,Country,Year,Pob
96,"Hong Kong SAR, China",1960,"3,075,605.0"
111,Ireland,1960,"2,828,600.0"
154,Mexico,1960,"37,771,861.0"
362,"Hong Kong SAR, China",1961,"3,168,100.0"
377,Ireland,1961,"2,824,400.0"
...,...,...,...
15805,Ireland,2019,"4,934,340.0"
15848,Mexico,2019,"127,575,529.0"
16056,"Hong Kong SAR, China",2020,"7,481,800.0"
16071,Ireland,2020,"4,994,724.0"


In [135]:
#   Ordenamos el Subconjunto por pais y año
df_ejemplo = df_ejemplo.set_index(['Country','Year']).sort_index()

In [137]:
df_ejemplo

Pob
Country              Year              
Hong Kong SAR, China 1960   3,075,605.0
                     1961   3,168,100.0
                     1962   3,305,200.0
                     1963   3,420,900.0
                     1964   3,504,600.0
...                                 ...
Mexico               2016 123,333,379.0
                     2017 124,777,326.0
                     2018 126,190,782.0
                     2019 127,575,529.0
                     2020 128,932,753.0

[183 rows x 1 columns]

In [ ]:
#   Seleccionamos un Pais y años especificos (Seleccion multi-index)
df_ejemplo.loc['Mexico'].loc['1997':'2020', :]

In [159]:
df_ejemplo.xs(key='1990', level='Year')

,Pob
Country,
"Hong Kong SAR, China","5,704,500.0"
Ireland,"3,513,974.0"
Mexico,"83,943,135.0"


# Categorizamos el DataSet

In [163]:
df_paises = df_pob.set_index(['Country', 'Year']).sort_index(ascending=[True, True])

In [164]:
df_paises

Pob
Country     Year             
Afghanistan 1960  8,996,967.0
            1961  9,169,406.0
            1962  9,351,442.0
            1963  9,543,200.0
            1964  9,744,772.0
...                       ...
Zimbabwe    2016 14,030,338.0
            2017 14,236,599.0
            2018 14,438,812.0
            2019 14,645,473.0
            2020 14,862,927.0

[16226 rows x 1 columns]

In [170]:
ids = pd.IndexSlice
df_paises.loc[ids['Macedonia':'Mexico', '2000':'2020'],:].sort_index()

Pob
Country    Year              
Madagascar 2000  15,766,806.0
           2001  16,260,933.0
           2002  16,765,122.0
           2003  17,279,139.0
           2004  17,802,992.0
...                       ...
Mexico     2016 123,333,379.0
           2017 124,777,326.0
           2018 126,190,782.0
           2019 127,575,529.0
           2020 128,932,753.0

[210 rows x 1 columns]